In [1]:
import pandas as pd

from autoqchem.molecule import molecule
from autoqchem.pbs_manager import pbs_manager
from autoqchem.helper_classes import pbs_status
from autoqchem.draw_utils import draw


## pbs_manager
### autoQchem job을 관리하는 manager. 누리온 서버에 접속하여 job 생성 및 upload, 삭제 등 job을 관리한다

In [2]:
### job_file은 pbs_manager.pkl의 prefix로 사용합니다. 
### project 혹은 사용자별로 job 관리가 필요하다면 job_file을 입력해주세요. 
### job_file 이 정의되어 있지 않으면 pbs_manager.pkl을 default로 사용합니다. 

pm=pbs_manager(user='q632a02', 
               host='nurion.ksc.re.kr', 
               sftp_host='nurion-dm.ksc.re.kr', 
               workdir='/bo/workspace/nurion',
              #job_file='q632a02'
              ) 

In [14]:
pm.connect()
## OTP가 필요. password = PDL4230+-

In [3]:
### Set molecule information
### 여러 개의 molecule을 , 로 연결

my_smiles=[
    "FC(C1=COCCN1CC2=C(C=CC=C3)C3=CC=C2)(F)F"    
]

In [4]:
mols = []
for smile in my_smiles:
    mols.append(molecule(smile, num_conf=5))

# Gaussian Job setup
### Gaussian Job 생성 조건

In [5]:
### solvent list : https://gaussian.com/scrf/?tabid=7. 테이블에서 실제 실험 사용할 조건을 선택 기재

for mol in mols:
    pm.create_jobs_for_molecule(mol, light_basis_set='6-31G', solvent='None', wall_time='06:00:00')

In [3]:
pm.get_job_stats(split_by_can=True)

status,uploaded
can,
FC(C1=COCCN1CC2=C(C=CC=C3)C3=CC=C2)(F)F,4


## Job 실행
job status가 create 인 job을 실행시키는 작업으로 nurion 서버에 queue가 걸리게 됩니다.\
실제 job을 실행시킬 때 실행하세요.

Job 실행을 위해서는 job file이 넘어갈 수 있도록 pm.connect_sftp() 실행이 필요합니다

In [7]:
pm.connect_sftp()

## OTP 연결 필요

Password(OTP):  ········
Password:  ········


In [8]:
### job을 nurion에 제출. 제출할 때만 실행시키세요. 

pm.submit_jobs()

Password(OTP):  ········
Password:  ········


# Job 상태 조회
서버에 현재 실행 중이거나 대기 중인 작업을 조회하는 기능입니다. \
job 목록에서 "S"가 status를 의미하며 아래와 같이 상태를 표기합니다 \
Q : Queue 대기 중 \
R : Run 실행 중

In [11]:
pm.retrieve_jobs()  

There are 0 running/pending jobs, 10 finished jobs.


In [12]:
pm.get_job_stats(split_by_can=True)

status,done
can,
FC(C1=COCCN1CC2=C(C=CC=C3)C3=CC=C2)(F)F,10


## 결과 DB insert
위의 job이 모두 끝나면 계산 결과를 DB에 넣는 기능입니다. \
현재 노트북에 설치된 mongodb에 넣게 되며 tag를 지정하여 넣으면 추후 해당 tag set을 조회하는데 유용합니다.

In [13]:
### JOB 이 끝난 것을 확인 후 TAG 명을 지정 후 upload. 
pm.upload_done_molecules_to_db(tags=["GFTE_subs"])

('1bc82e096ff1eeb6ff167e3f7b29e629', '5fb5fc7b21a4b318ed62c982985cf869', 'f041a9cc3bc15393f7c15c33fbc4c79b', '90bf1799623110f6216a7a5d06df9af5', 'bd92df0847207211c94accfd9b225666', 'e4c8bb344261a5ea1da8a2079d1b1fd3', 'ca89caa56869abd1b0156fa2ecf7a2ff', '67ab5c30113082258c72d0fa0443a463', '81f86574620386bf4637e934158c814d', 'cf63f2b48afb3485e1f4469f5773aaf5')
True
4
